In [1]:
#Download file from MTA
#Input into a PD Dataframe (e.g. correctly label columns)
#Helper tables: Define MTA station Mapping
#reformat PD into a usable DF - Unit ID, Date-time, station name, entries, exits
#sort on unit-id (will sort by time too). remove duplicates. [possibly find a better method]

#- Which station has the most number of units? ... Penn Station (unique turnstile units for the Week of Feb 2nd)
#- What is the total number of entries & exits across the subway system for February 1, 2013? ...neet to calculate net entries and exits [2hrs]
#- Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date? ... add busyness column. filter for only feb 1. group by station [30 mins]
#- What stations have seen the most usage growth/decline in 2013? [NEEDS clarification - since daily variance is significant] ...need to download whole dataset for 2012 and 2013. (speed up calculation) [3hrs]
#- What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely? ... [2hrs]
#- Bonus:  What hour is the busiest for station CANAL ST in Q1 2013?

#volume by day by station, by turnstile

import pandas as pd
from datetime import datetime


#Questions
#1) What kind of assumptions can I make about the data (e.g. can I assume counts are accurate?)
#2) What does "door open logon lgf man..." mean?
#3) Does question 1 refer to number of units on Feb 1?
#4) I'm making the assumption about day-cutoffs. credit it based on the day the 4 hour cutof ends.

In [2]:
raw_MTA_data = pd.read_csv("~/Projects/NYC_MTA_DATA_ANALYSIS/data/turnstile_130202.txt")
raw_MTA_data.columns= ["C/A","UNIT","SCP","DATE1","TIME1","DESC1","ENTRIES1","EXITS1","DATE2","TIME2","DESC2","ENTRIES2","EXITS2","DATE3",
"TIME3","DESC3","ENTRIES3","EXITS3","DATE4","TIME4","DESC4","ENTRIES4","EXITS4","DATE5","TIME5","DESC5","ENTRIES5","EXITS5","DATE6",
"TIME6","DESC6","ENTRIES6","EXITS6","DATE7","TIME7","DESC7","ENTRIES7","EXITS7","DATE8","TIME8","DESC8","ENTRIES8","EXITS8"]
print(raw_MTA_data.head())

    C/A  UNIT       SCP     DATE1     TIME1    DESC1  ENTRIES1   EXITS1  \
0  A002  R051  02-00-00  01-27-13  11:00:00  REGULAR   3967826  1367578   
1  A002  R051  02-00-00  01-28-13  19:00:00  REGULAR   3969685  1368236   
2  A002  R051  02-00-00  01-30-13  03:00:00  REGULAR   3971424  1368820   
3  A002  R051  02-00-00  01-31-13  11:00:00  REGULAR   3973086  1369749   
4  A002  R051  02-00-00  02-01-13  15:00:00  REGULAR   3974913  1370326   

      DATE2     TIME2  ...     DATE7     TIME7    DESC7   ENTRIES7     EXITS7  \
0  01-27-13  15:00:00  ...  01-28-13  11:00:00  REGULAR  3968601.0  1368084.0   
1  01-28-13  23:00:00  ...  01-29-13  19:00:00  REGULAR  3971170.0  1368796.0   
2  01-30-13  07:00:00  ...  01-31-13  03:00:00  REGULAR  3972902.0  1369398.0   
3  01-31-13  14:43:33  ...  02-01-13  07:00:00  REGULAR  3974518.0  1369972.0   
4  02-01-13  19:00:00  ...       NaN       NaN      NaN        NaN        NaN   

      DATE8     TIME8    DESC8   ENTRIES8     EXITS8  
0  01-2

In [77]:
#create MTA_turnstiles df... unit, date-time, entries_raw, exits_raw
#Loop through raw dataset. for each row, make an entry in the MTA turnstile data set.
raw_MTA_data_tmp = raw_MTA_data.copy()
raw_MTA_data_tmp["Unit_ID"] = raw_MTA_data_tmp["C/A"] + raw_MTA_data_tmp["UNIT"] + raw_MTA_data_tmp["SCP"]
raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["C/A", "SCP"],axis=1)

def addTimeStamp(vec):
    x = str(vec[0])
    y = str(vec[1])
    try:
        return datetime.strptime(x + " " + y, "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S")
    except:
        return "NULL"

for i in range(1,8+1):
    raw_MTA_data_tmp["Datetime_" + str(i)] = raw_MTA_data_tmp[["DATE" + str(i), "TIME" + str(i)]].apply(addTimeStamp,axis=1)

#raw_MTA_data_tmp["Date_time_1"] = raw_MTA_data_tmp.apply(lambda x: datetime.strptime(x["DATE1"] + " " + x["TIME1"], "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S"),axis=1) 
raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["DATE1", "TIME1", "DATE2", "TIME2", "DATE3", "TIME3", "DATE4", "TIME4", "DATE5", "TIME5", "DATE6", "TIME6", "DATE7", "TIME7", "DATE8", "TIME8"],axis=1)
print(raw_MTA_data_tmp.head())






   UNIT    DESC1  ENTRIES1   EXITS1    DESC2   ENTRIES2     EXITS2    DESC3  \
0  R051  REGULAR   3967826  1367578  REGULAR  3967988.0  1367633.0  REGULAR   
1  R051  REGULAR   3969685  1368236  REGULAR  3969945.0  1368263.0  REGULAR   
2  R051  REGULAR   3971424  1368820  REGULAR  3971442.0  1368857.0  REGULAR   
3  R051  REGULAR   3973086  1369749  REGULAR  3973280.0  1369836.0  REGULAR   
4  R051  REGULAR   3974913  1370326  REGULAR  3975747.0  1370378.0  REGULAR   

    ENTRIES3     EXITS3  ...     EXITS8           Unit_ID         Datetime_1  \
0  3968260.0  1367679.0  ...  1368165.0  A002R05102-00-00  13-01-27 11:00:00   
1  3969947.0  1368267.0  ...  1368820.0  A002R05102-00-00  13-01-28 19:00:00   
2  3971586.0  1369179.0  ...  1369435.0  A002R05102-00-00  13-01-30 03:00:00   
3  3973287.0  1369836.0  ...  1370294.0  A002R05102-00-00  13-01-31 11:00:00   
4  3976035.0  1370406.0  ...        NaN  A002R05102-00-00  13-02-01 15:00:00   

          Datetime_2         Datetime_3     

In [97]:
#add station name & count number of turnstiles per station
remote_station_mapping = pd.read_excel("../data/Remote-Booth-Station.xls")
remote_station_mapping["UNIT"] = remote_station_mapping["Remote"]

raw_MTA_data_tmp2 = raw_MTA_data_tmp[["UNIT", "Unit_ID"]].merge(remote_station_mapping[["UNIT", "Station"]], on=["UNIT"])

MTA_turnstiles_unique = raw_MTA_data_tmp2.drop_duplicates(subset=["Unit_ID"])
x = MTA_turnstiles_unique.groupby(by="Station").agg(["count"])["UNIT"]
x = x.sort_values(by="count")
print(x)
#print(MTA_turnstiles.groupby(by=["Station"]).count())

                 count
Station               
CLEVELAND ST         2
ZEREGA AVE           3
BEACH 44 ST          3
BEACH 36 ST          3
BEACH 25 ST          3
...                ...
CANAL ST            56
42 ST-GRD CNTRL     61
NEWARK HW BMEBE     64
FULTON ST           77
34 ST-PENN STA     100

[362 rows x 1 columns]


In [100]:
unique_stations = remote_station_mapping.drop_duplicates(subset="Station")


In [76]:
MTA_turnstiles = pd.DataFrame(columns = ["Unit", "Datetime", "Station", "Cumulative entries", "Cumulative exits", "Net entries", "Net exits"])

#runtime - 10 minutes
for i in range(int(len(raw_MTA_data_tmp)/10)):
    for j in range(1,8+1):
        index = i*(j-1)+j
        MTA_turnstiles.loc[index,"Unit"] = raw_MTA_data_tmp.loc[i,"Unit_ID"]
        MTA_turnstiles.loc[index,"Datetime"] = raw_MTA_data_tmp.loc[i,"Datetime_" + str(j)]
        MTA_turnstiles.loc[index,"Station"] = raw_MTA_data_tmp.loc[i,"Station"]
        MTA_turnstiles.loc[index,"Cumulative entries"] = raw_MTA_data_tmp.loc[i,"ENTRIES" + str(j)]
        MTA_turnstiles.loc[index,"Cumulative exits"] = raw_MTA_data_tmp.loc[i,"EXITS" + str(j)]
        MTA_turnstiles.loc[index,"Net entries"] = -1
        MTA_turnstiles.loc[index,"Net exits"] = -1

MTA_turnstiles_backup = MTA_turnstiles.copy(deep=True)


KeyboardInterrupt: 

In [13]:
#sort & make ID the index...
MTA_turnstiles["ID"] = MTA_turnstiles["Unit"] + " " + MTA_turnstiles["Datetime"] 
MTA_turnstiles.sort_values(by=["ID"])

,Unit,Datetime,Station,Cumulative entries,Cumulative exits,Net entries,Net exits,ID
2,A002R05102-00-00,13-01-27 15:00:00,,3967988.0,1367633.0,-1,-1,A002R05102-00-00 13-01-27 15:00:00
3,A002R05102-00-00,13-01-28 23:00:00,,3969945.0,1368263.0,-1,-1,A002R05102-00-00 13-01-28 23:00:00
4,A002R05102-00-00,13-01-30 07:00:00,,3971442.0,1368857.0,-1,-1,A002R05102-00-00 13-01-30 07:00:00
5,A002R05102-00-00,13-01-31 14:43:33,,3973280.0,1369836.0,-1,-1,A002R05102-00-00 13-01-31 14:43:33
6,A002R05102-00-00,13-02-01 19:00:00,,3975747.0,1370378.0,-1,-1,A002R05102-00-00 13-02-01 19:00:00
...,...,...,...,...,...,...,...,...
9259,C009R05703-00-02,13-01-26 15:00:00,,4176456.0,4589186.0,-1,-1,C009R05703-00-02 13-01-26 15:00:00
12345,C009R05703-00-02,13-01-26 19:00:00,,4176709.0,4589647.0,-1,-1,C009R05703-00-02 13-01-26 19:00:00
15431,C009R05703-00-02,13-01-26 23:00:00,,4176849.0,4589934.0,-1,-1,C009R05703-00-02 13-01-26 23:00:00
18517,C009R05703-00-02,13-01-27 03:00:00,,4176905.0,4590049.0,-1,-1,C009R05703-00-02 13-01-27 03:00:00


                  Datetime  Station  Cumulative entries  Cumulative exits  \
Unit                                                                        
A002R05102-00-00         5        3                   5                 5   
A002R05102-00-01         6        5                   6                 6   
A002R05102-03-00         7        4                   7                 7   
A002R05102-03-01         8        2                   8                 8   
A002R05102-03-02         7        3                   7                 7   
...                    ...      ...                 ...               ...   
C008R09900-06-01        40        0                  39                37   
C008R09900-06-02        40        0                  39                38   
C009R05703-00-00        47        0                  40                40   
C009R05703-00-01        41        0                  39                39   
C009R05703-00-02         8        0                   8                 8   

In [ ]:
MTA_turnstiles